# AWS Summarization Deployment
This notebook includes both Lambda-based and SageMaker-based deployments for summarization.

## 1. Extractive Summarization on Lambda using DistilBERT

In [ ]:
from transformers import pipeline
from fastapi import FastAPI
from pydantic import BaseModel

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

app = FastAPI()

class Article(BaseModel):
    text: str

@app.post("/summarize/")
async def summarize(article: Article):
    result = summarizer(article.text, max_length=120, min_length=30, do_sample=False)
    return {"summary": result[0]["summary_text"]}


## 2. Abstractive Summarization with T5 on SageMaker

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("xsum")
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def preprocess(example):
    input_text = "summarize: " + example["document"]
    model_inputs = tokenizer(input_text, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(example["summary"], max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized = dataset.map(preprocess, batched=True)

args = TrainingArguments(
    output_dir="./t5-summarization",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_total_limit=1,
)
trainer = Trainer(model=model, args=args, train_dataset=tokenized["train"])
trainer.train()

model.save_pretrained("t5-summarizer")
tokenizer.save_pretrained("t5-summarizer")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

def model_fn(model_dir):
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir)
    return {"model": model, "tokenizer": tokenizer}

def predict_fn(input_data, model_obj):
    text = input_data["text"]
    tokenizer = model_obj["tokenizer"]
    model = model_obj["model"]

    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=2)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return {"summary": summary}
